In [23]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown,display


In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")  


OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AI


In [3]:
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

openai = OpenAI(api_key=openai_api_key)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)


In [5]:
GEMINI_MODEL = "gemini-2.5-flash-lite"
OPENAI_MODEL = "gpt-5"

In [7]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '24.5.0',
  'version': 'Darwin Kernel Version 24.5.0: Tue Apr 22 19:54:43 PDT 2025; root:xnu-11417.121.6~2/RELEASE_ARM64_T8132',
  'kernel': '24.5.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin24.5.0'},
 'package_managers': ['xcode-select (CLT)'],
 'cpu': {'brand': 'Apple M4',
  'cores_logical': 10,
  'cores_physical': 10,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 17.0.0 (clang-1700.0.13.5)',
   'g++': 'Apple clang version 17.0.0 (clang-1700.0.13.5)',
   'clang': 'Apple clang version 17.0.0 (clang-1700.0.13.5)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [8]:
message = f"""
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

You’re already set up. Your system has Apple Clang (C/C++ compiler) from the Xcode Command Line Tools:
- clang/clang++: Apple clang version 17.0.0
No installation is needed to compile and run a single C++ file.

For fastest runtime performance on your Apple Silicon (M4), use clang++ with optimization, ThinLTO, and native CPU tuning. In Python, set:

compile_command = ["clang++", "-std=c++20", "-O3", "-DNDEBUG", "-flto=thin", "-mcpu=native", "main.cpp", "-o", "main"]
run_command = ["./main"]

This will produce an optimized binary named “main” and then execute it.

In [9]:
compile_command = ["clang++", "-std=c++17", "-Ofast", "-mcpu=native", "-flto=thin", "-fvisibility=hidden", "-DNDEBUG", "main.cpp", "-o", "main"]
run_command = ["./main"]

In [10]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
The system information is:
{system_info}
Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
{compile_command}
Respond only with C++ code.
Python code to port:

```python
{python}
```
"""

In [18]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [12]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [19]:
def port(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [14]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [ ]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)


    # run string as python code

In [16]:
run_python(pi)

Result: 3.141592656089
Execution Time: 10.419234 seconds


In [20]:

port(openai,OPENAI_MODEL,pi)

In [21]:
def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [24]:
compile_and_run()

Result: 3.141592656777
Execution Time: 0.089457 seconds

Result: 3.141592656777
Execution Time: 0.056746 seconds

Result: 3.141592656777
Execution Time: 0.054609 seconds

